## 1.课堂code复现

In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread('./lenna.jpg')
cv2.imshow('lenna', img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

####  Gaussian kernel ---- 1 derivative 

In [5]:
# Gaussian Kernel Effect
g_img = cv2.GaussianBlur(img,(7,7),5)# kernel variance
cv2.imshow('gaussian_blur_lenna', g_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

In [6]:
# 图像变更模糊，因为范围更大，平均效果更明显
g_img = cv2.GaussianBlur(img,(17,17),5)
cv2.imshow('gaussian_blur_lenna', g_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

# 图像更清晰，因为方差更小，高斯图像更尖锐，中心点起的作用更大
g_img = cv2.GaussianBlur(img,(7,7),1)
cv2.imshow('gaussian_blur_lenna', g_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

In [7]:
# 来看看高斯核
kernel = cv2.getGaussianKernel(7, 5)
print(kernel)

[[0.12895603]
 [0.14251846]
 [0.15133131]
 [0.1543884 ]
 [0.15133131]
 [0.14251846]
 [0.12895603]]


In [8]:
# 为啥一维，因为一维运算快
# 理论解释
# 用显式地代码看隐式地高斯和显示地分步高斯地效果
g1_img = cv2.GaussianBlur(img,(7,7),5)
g2_img = cv2.sepFilter2D(img, -1, kernel, kernel) # ori, depth, kernelX, kernelY
cv2.imshow('g1_blur_lenna', g1_img)
cv2.imshow('g2_blur_lenna', g2_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

#### 2nd derivative: laplacian （双边缘效果）

In [11]:
# 双边缘
kernel_lap = np.array([[0,1,0],[1,-4,1],[0,1,0]],np.float32)
lap_img = cv2.filter2D(img,-1,kernel = kernel_lap)
cv2.imshow('lap_lenna', lap_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

In [19]:
kernel_sharp = np.array([[0, 1, 0], [1, -3, 1], [0, 1, 0]], np.float32) 
lap_img = cv2.filter2D(img, -1, kernel=kernel_sharp)
cv2.imshow('sharp_lenna', lap_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()
# 这样不对，因为，周围有4个1，中间是-3，虽然有边缘效果，但是周围得1会使得原kernel有滤波效果，使图像模糊；

In [20]:
# 解决：所以取kernel_lap得相反数，再加上原图像，这样突出了中心像素，效果类似于小方差的高斯，所以可以既有边缘效果，又保留图像清晰度
kernel_sharp = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32) 
lap_img = cv2.filter2D(img, -1, kernel=kernel_sharp)
cv2.imshow('sharp_lenna', lap_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()
#类似锐化效果

In [21]:
# 更“凶猛”的边缘效果
# 不仅考虑x-y方向上的梯度，同时考虑了对角线方向上的梯度
kernel_sharp = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], np.float32) 
lap_img = cv2.filter2D(img, -1, kernel=kernel_sharp)
cv2.imshow('sharp_lenna', lap_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

In [22]:
# 边缘检测
# x轴
edgex = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], np.float32)
sharp_img = cv2.filter2D(img, -1, kernel=edgex)
cv2.imshow('edgex_lenna', sharp_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

# y轴
edgey = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
sharpy_img = cv2.filter2D(img, -1, kernel=edgey)
cv2.imshow('edgex_lenna', sharp_img)
cv2.imshow('edgey_lenna', sharpy_img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

#### 角点

In [ ]:
img = cv2.imread('./board.jpg')
img = cv2.resize(img, (640, 480))
img_gray = np.float32(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
print(img_gray)

img_harris = cv2.cornerHarris(img_gray, 2, 3, 0.05)
cv2.imshow('img_harris ', img_harris)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()
# 没法看原因：1. float类型； 2. img_harris本质上是每个pixel对于Harris函数的响应值
# 没有看的价值
print(img_harris)

# 为了显示清楚
# img_harris = cv2.dilate(img_harris , None)

thres = 0.05 * np.max(img_harris)
img[img_harris > thres] = [0, 0, 255]
cv2.imshow('img_harris ', img)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

#### SIFT 

In [ ]:
img = cv2.imread('lenna.jpg')
# create sift class
sift = cv2.xfeatures2d.SIFT_create()
# detect SIFT
kp = sift.detect(img,None)   # None for mask
# compute SIFT descriptor 检测关键点和对应的描述子
kp,des = sift.compute(img,kp)
print(des.shape)
img_sift= cv2.drawKeypoints(img,kp,outImage=np.array([]), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('lenna_sift.jpg', img_sift)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

## 2.median blur

 Finish 2D convolution/filtering by your self. 
What you are supposed to do can be described as "median blur", which means by using a sliding window on an image, your task is not going to do a normal convolution, but to find the median value within that crop.

You can assume your input has only one channel. (a.k.a a normal 2D list/vector) And you do need to consider the padding method and size. There are 2 padding ways: REPLICA & ZERO. When "REPLICA" are given to you, the padded pixels are the same with the border pixels. E.g is [1 2 3] is your image, the padded version will be [(...1 1) 1 2 3 (3 3...)] where how many 1 & 3 in the parenthesis depends on your padding size. When "ZERO", the padded version will be [(...0 0) 1 2 3 (0 0...)]

Assume your input's size of the image is W x H, kernel size's m x n. You may first complete a version with O(W·H·m·n log(m·n)) to O(W·H·m·n·m·n)). 

Follow up 1: Can it be completed in a shorter time complexity?

Follow up 2: Can it be completed in O(W·H·m·n)?

Python version:
def medianBlur(img, kernel, padding_way):

    img & kernel is List of List; padding_way a string

Please finish your code under this blank

In [1]:
import cv2
import numpy as np
import math

In [8]:
def medianBlur(img,kernel,padding_way):
    s = list(kernel)
    m,n = s[0],s[1]
    img_new = img
    if padding_way =="REPLICA":
        img = np.array(img)
        img = np.pad(img,((math.ceil((m-1)/2),math.floor((n-1)/2)),(math.ceil((m-1)/2),math.floor((n-1)/2))),'edge')
    if padding_way =="ZERO":
        img = np.array(img)
        img = np.pad(img,((math.ceil((m-1)/2),math.floor((n-1)/2)),(math.ceil((m-1)/2),math.floor((n-1)/2))),'constant')
        
    W = img.shape[0]
    H = img.shape[1]
    i = 0
    while i <= W-m:
        j = 0
        while j<=H-n:
            img_new[i][j] = np.median(img[i:i+m-1,j:j+n-1]).astype(img.dtype)
            j+=1
        i+=1
    return img_new

In [14]:
img = cv2.imread('./lenna.jpg', 0)
cv2.imshow('img', img)
#img = img[100:200, 100:200]
img_blur_1 = medianBlur(img, (10,10), 'ZERO')
img_blur_2 = medianBlur(img, (10,10), 'REPLICA')
cv2.imshow('img_blur_1', img_blur_1)
cv2.imshow('img_blur_2', img_blur_2)
key = cv2.waitKey()
if key == 27:
    cv2.destroyAllWindows()

## 3.Reading + RANSAC algorithm


We haven't told RANSAC algorithm this week. So please try to do the reading.And now, we can describe it here:

We have 2 sets of points, say, Points A and Points B. We use A.1 to denote the first point in A, B.2 the 2nd point in B and so forth. Ideally, A.1 is corresponding to B.1, ... A.m corresponding B.m. However, it's obvious that the matching cannot be so perfect and the matching in our real world is like: A.1-B.13, A.2-B.24, A.3-x (has no matching), x-B.5, A.4-B.24(This is a wrong matching) ...

The target of RANSAC is to find out the true matching within this messy.

Algorithm for this procedure can be described like this:
1. Choose 4 pair of points randomly in our matching points. Those four called "inlier" (中文： 内点) while  others "outlier" (中文： 外点)
2. Get the homography of the inliers
3. Use this computed homography to test all the other outliers. And seperated them by using a threshold into two parts:

 a. new inliers which is satisfied our computed homography
 
 b. new outliers which is not satisfied by our computed homography.


4. Get our all inliers (new inliers + old inliers) and goto step 2
5. As long as there's no changes or we have already repeated step 2-4 k, a number actually can be computed, times, we jump out of the recursion. The final homography matrix will be the one that we want.

[WARNING!!! RANSAC is a general method. Here we add our matching background to that.]

##### Your task: please complete pseudo code (it would be great if you hand in real code!) of this procedure.
Python:

def ransacMatching(A, B):

A & B: List of List

In [2]:
import cv2
import numpy as np
import random

In [70]:
def calculate_homo(cor_p):  #计算一个平面到另外一个平面的投影映射矩阵
    #Get the homography of the inliers
    alist = []# 存储系数矩阵
    for corr in cor_p:
        p1 = np.mat([corr.item(0), corr.item(1), 1])
        p2 = np.mat([corr.item(2), corr.item(3), 1])
        a1 = [-p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2), 0, 0, 0,p2.item(0) * p1.item(0), p2.item(0) * p1.item(1), p2.item(0) * p1.item(2)]
        a2 = [0, 0, 0, -p2.item(2) * p1.item(0), -p2.item(2) * p1.item(1), -p2.item(2) * p1.item(2),p2.item(1) * p1.item(0), p2.item(1) * p1.item(1), p2.item(1) * p1.item(2)]
        alist.append(a1)
        alist.append(a2)
    matrix_a = np.mat(alist) # 变成矩阵形式 matrix_a 是系数矩阵
    u, s, v = np.linalg.svd(matrix_a) #奇异值分解
    h = np.reshape(v[8], (3, 3)) #选取最小的右奇异值，将其变成3*3矩阵
    h = (1 / h.item(8)) * h # 使单应性矩阵最后一个值为1
    return h

In [80]:
def geometricDistance(corresponse, h):
    p1 = np.transpose(np.mat([corresponse[0].item(0), corresponse[0].item(1), 1]))
    p2 = np.transpose(np.mat([corresponse[0].item(2), corresponse[0].item(3), 1]))

    diff = p2 - np.dot(h, p1) # p1 in queryIdx * h 映射回trainIdx 和 p2 in trainIdx 求距离
    return np.linalg.norm(diff)

instruction of cv2.BFMatcher 
https://www.jianshu.com/p/ed57ee1056ab

In [81]:
def ransacMatching(A,B):
    matcher = cv2.BFMatcher(cv2.NORM_L2, True)
    matches = matcher.match(A[1], B[1])
    cor_p = [] #存放匹配点
    for match in matches:
        (x1, y1) = A[0][match.queryIdx].pt  #(x1,y1) is coordinate of match point in A----queryIdx
        (x2, y2) = B[0][match.trainIdx].pt  #(x2,y2) is coordinate of match point in B----trainIdx
        cor_p.append([x1, y1, x2, y2])
    cor_p = np.mat(cor_p)
    maxInliers = []
    finalH = None
    inliers = []
    
    #randomly choose 4 matches
    for j in range(4): 
        tem = random.randrange(0,len(cor_p))
        inliers.append(cor_p[tem])
        
    outliers = cor_p
    
    for i in range(1000):   #迭代求取最大的inliers和 单应性矩阵
        h = calculate_homo(inliers)
        
        for k in range(len(outliers)):  #计算每个局外点与单应性矩阵的距离，对局外点进行分类
            
            d = geometricDistance(outliers[k], h)
            if d < 5:
                inliers.append(outliers[k])
                
        if len(inliers) > len(maxInliers):
            maxInliers = inliers
            finalH = h
            
        if len(inliers) == len(maxInliers):  # 当局内点数量不再增加，跳出循环
            break
            
    return finalH, maxInliers, matches

In [82]:
def drawMatches(img1, img2, kp1, kp2, matches, inliers):
    rows1 = img1.shape[0]
    cols1 = img1.shape[1]
    rows2 = img2.shape[0]
    cols2 = img2.shape[1]

    out = np.zeros((max(rows1, rows2), cols1 + cols2, 3), dtype='uint8')
    out[0:rows1, 0:cols1, :] = img1
    out[0:rows2, cols1:, :] = img2

    for i in matches:
        img1_idx = i.queryIdx
        img2_idx = i.trainIdx
        (x1, y1) = kp1[img1_idx].pt
        (x2, y2) = kp2[img2_idx].pt

        if inliers is not None:
            cv2.line(out, (int(x1), int(y1)), (int(x2)+cols1, int(y2)), (0, 255, 0), 1)

        if inliers is None:
            cv2.line(out, (int(x1), int(y1)), (int(x2)+cols1, int(y2)), (255, 0, 0), 1)
    return out

In [83]:
def run():
    # read an image
    img1 = cv2.imread('img1.png')
    img2 = cv2.imread('img2.png')

    # find key point and descriptors of img1
    sift1 = cv2.xfeatures2d.SIFT_create()
    kp1 = sift1.detect(img1, None)
    kp1, des1 = sift1.compute(img1, kp1)

    # find key point and descriptors of img2
    sift2 = cv2.xfeatures2d.SIFT_create()
    kp2 = sift2.detect(img2, None)
    kp2, des2 = sift2.compute(img2, kp2)

    A = []
    B = []
    A.append(kp1)
    A.append(des1)
    B.append(kp2)
    B.append(des2) #A[0] and B[0] keypoint/A[1] and B[1] descriptors each descriptor has 128 dimensions
    finalH, MaxInliers, matches = ransacMatching(A,B)
    imgout = drawMatches(img1, img2, kp1, kp2, matches, MaxInliers)
    print('final homography', finalH)
    print('final inliers count', len(MaxInliers))
    cv2.imwrite('matchimg.png', imgout)
    cv2.imshow('imgout', imgout)
    cv2.waitKey()

In [84]:
run()

final homography [[ 5.63590619e-02 -9.83063396e-01  2.07603806e+02]
 [ 5.09760701e-02 -8.33847761e-01  1.75033563e+02]
 [ 3.54802300e-04 -4.84296684e-03  1.00000000e+00]]
final inliers count 4
